In [1]:
# Pip install SDK
!pip install --force-reinstall --no-cache fubon_neo-2.0.1-cp37-abi3-win_amd64.whl

Processing e:\文件\github\2024_09_22_chihlee_sql\fubon\fubon_neo-2.0.1-cp37-abi3-win_amd64.whl
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2


In [1]:
import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

import fubon_neo
fubon_neo.__version__

 # 匯入 SDK Library
from fubon_neo.sdk import FubonSDK, Order
from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction

sdk = FubonSDK()

# 連接到 fubon API
# accounts = sdk.login   (id              = os.environ    ['FUBON_ID'],
#                        id_password     = os.environ    ['FUBON_PW'],
#                        credential_path = os.environ    ['FUBON_CREDENTIAL_PATH'],
#                        path_password   = os.environ    ['FUBON_PATH_PASSWORD'])

# accounts = sdk.login   (os.environ  ['FUBON_ID'],
#                        os.environ  ['FUBON_PW'],
#                        os.environ  ['FUBON_CREDENTIAL_PATH'],
#                        os.environ  ['FUBON_PATH_PASSWORD'])
# 從環境變數中讀取參數
id              = os.getenv("FUBON_ID")
id_password     = os.getenv("FUBON_PASSWORD")
credential_path = os.getenv("FUBON_CREDENTIAL_PATH")
path_password   = os.getenv("FUBON_PATH_PASSWORD")
print(id, id_password, credential_path, path_password)
# 驗證所有必要的參數是否存在
if None in (id, id_password, credential_path, path_password):
    print(f"帳號/密碼有誤！")

accounts = sdk.login(id, id_password, credential_path, path_password)   # 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
print(accounts)

sdk.init_realtime() # 建立行情元件連線

import json
import traceback

subscribe_ids = []  # 訂閱頻道 id 列表

def handle_connect():  # 連線成功 callback
    print("行情連接成功")

def handle_disconnect(code, message):  # 連接斷線 callback
    print(f"行情連接斷線: {code}, {message}")

def handle_message(message): # 處理接收訊息 callback
    try:
        msg = json.loads(message)
        event = msg["event"]
        data = msg["data"]
    
        if event == "pong":
            return
        
        if event == "subscribed":
            id = data["id"]
            
            if id in subscribe_ids:
                print(f"Error: 訂閱 id {id} 已存在列表中")
            else:
                subscribe_ids.append(id)
    
        elif event == "unsubscribed":
            id = data["id"]
            
            try:
                subscribe_ids.remove(id)
            except:
                print(f"Error: 查無此筆訂閱 id 資料, id {id}")
    
        print(f'market data message: {message}')           
        
    except Exception as e:
        handle_error(f'Error parsing JSON: {e}', traceback.format_exc())

def handle_error(error,traceback_info=None):  # 處理程式錯誤訊息 callback
    print(f'market data error: {error}')
    if traceback_info:
        print(f'Traceback:\n{traceback_info}')

stock = sdk.marketdata.websocket_client.stock
stock.on("connect", handle_connect)
stock.on("message", handle_message)
stock.on("disconnect", handle_disconnect)
stock.on("error", handle_error)

stock.connect()  # WebSocket 連線

# 建立行情查詢 WebAPI 連線 Object Instance
restStock = sdk.marketdata.rest_client.stock

# 股票即時報價（依代碼查詢）
result = restStock.intraday.quote(symbol="2330")
stock_name = result.get('name', '未找到股票名稱')  # 使用 get() 方法獲取股票名稱
# print(result)
print(f"股票名稱: {stock_name}")
print(f"即時價格: {current_price}")


A121714564 tzj1nez2wyyy C:\CAFubon\A121714564/A121714564.pfx tzj1nez2wyyy
Result {
  is_success: True,
  message: None,
  data: [Account {
    name: "李定洲",
    branch_no: "20505",
    account: "37089",
    account_type: "stock",
}]
}
行情連接成功
market data message: {"event":"authenticated","data":{"message":"Authenticated successfully"}}
股票名稱: 台積電


NameError: name 'current_price' is not defined

market data message: {"event":"heartbeat","data":{"time":1732193400014964}}
market data message: {"event":"heartbeat","data":{"time":1732193430005758}}
market data message: {"event":"heartbeat","data":{"time":1732193460008469}}


In [2]:
import fubon_neo.sdk
help(fubon_neo.sdk)
help(FubonSDK)
help(Order)


Help on module fubon_neo.sdk in fubon_neo:

NAME
    fubon_neo.sdk

CLASSES
    builtins.CoreSDK(builtins.object)
        FubonSDK
    builtins.object
        MarketData
    
    class FubonSDK(builtins.CoreSDK)
     |  FubonSDK(url=None)
     |  
     |  fubon sdk for api trading
     |  
     |  Method resolution order:
     |      FubonSDK
     |      builtins.CoreSDK
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  init_realtime(self, mode=<Mode.Speed: 2>)
     |      Initial market data and get authorised
     |  
     |  ----------------------------------------------------------------------
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from builtins.CoreSDK:
     |  